### Transformaciones: Para este MVP no necesitas perfección, ¡necesitas rapidez! ⏩ Vas a hacer estas, y solo estas, transformaciones a los datos:

- Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.

- Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

- Los valores nulos del campo release date deben eliminarse.

- De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

- Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, - cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

- Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

Nota: Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila.

Deberán desanidarlos para poder unirlos al dataset de nuevo y hacer alguna de las consultas de la API!. 
O bien buscar la manera de acceder a esos datos sin desanidarlos.



In [514]:
import pandas as pd
import numpy as np
import json
import re


In [515]:
# cargar dataset
df=pd.read_csv('./Dataset/movies_dataset.csv')

C:\Users\TheFercho\AppData\Local\Temp\ipykernel_9452\797142096.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('./Dataset/movies_dataset.csv')


In [516]:
df.shape

(45466, 24)

#### Verificamos si las columnas son categoricas o numericas para luego transformarlas en el caso que sea necesario.

In [517]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [518]:
df.loc[df['id']=='1997-08-20']

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [519]:
# Filtrar los registros que no son enteros en la columna 'id'
registros_no_enteros = df.loc[~df['id'].str.isdigit()]
registros_no_enteros
#En este caso, se utiliza la función str.isdigit() para verificar si cada valor de la columna 'id' 
# es un número entero. Luego, se nega el resultado utilizando el operador ~ para obtener los registros 
# que no son enteros. El resultado se asigna a la variable registros_no_enteros, y finalmente se imprime.

#Esto te proporcionará los registros que no son números enteros en la columna 'id', 
# los cuales podrías modificar o convertir a tipo integer según tus necesidades.

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [520]:
# Eliminar la fila donde la columna 'id' tiene el valor '1997-08-20'
df = df.drop(df[df['id'] == '1997-08-20'].index)

In [521]:
# Obtener los índices de los registros que no son enteros en la columna 'id'
indices_no_enteros = df[~df['id'].str.isdigit()].index

# Eliminar los registros que no son enteros
df = df.drop(indices_no_enteros)

#En este caso, se utiliza el método ~ junto con str.isdigit() para obtener los índices de las filas 
# que no son números enteros en la columna 'id'. Estos índices se asignan a la variable indices_no_enteros. 
# Luego, se utiliza el método drop junto con los índices para eliminar las filas correspondientes. 
# El resultado se asigna nuevamente al DataFrame df

In [522]:
df.id.astype(int)

0           862
1          8844
2         15602
3         31357
4         11862
          ...  
45461    439050
45462    111109
45463     67758
45464    227506
45465    461257
Name: id, Length: 45463, dtype: int32

#### - Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

In [523]:
df.drop(['video','imdb_id','adult','original_title','poster_path','homepage'],axis=1,inplace=True)
df.columns

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

#### - Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

In [524]:
df['revenue'].fillna(0,inplace=True)

In [525]:
df['budget'].fillna(0,inplace=True)

In [526]:
# verificamos valores nulos en general
df.isnull().sum()

belongs_to_collection    40972
budget                       0
genres                       0
id                           0
original_language           11
overview                   954
popularity                   3
production_companies         3
production_countries         3
release_date                87
revenue                      0
runtime                    260
spoken_languages             3
status                      84
tagline                  25051
title                        3
vote_average                 3
vote_count                   3
dtype: int64

In [527]:
# Mostrar los registros repetidos de la columna 'id'
duplicados = df[df.duplicated(['id'], keep=False)] # El parámetro keep=False se utiliza para marcar todos 
# los duplicados como True, de modo que se muestren todos los registros duplicados.
duplicados.head()

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
676,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",105045,de,"East-Berlin, 1961, shortly after the erection ...",0.122178,"[{'name': 'Studio Babelsberg', 'id': 264}, {'n...","[{'iso_3166_1': 'DE', 'name': 'Germany'}]",1995-02-16,0.0,115.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}]",Released,"A love, a hope, a wall.",The Promise,5.0,1.0
838,NaN,0,"[{'id': 18, 'name': 'Drama'}]",132641,ja,"Ten years into a marriage, the wife is disappo...",0.096079,"[{'name': 'Toho Company', 'id': 882}]","[{'iso_3166_1': 'JP', 'name': 'Japan'}]",1953-04-29,0.0,89.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Wife,0.0,0.0
949,NaN,4,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",22649,en,British nurse Catherine Barkley (Helen Hayes) ...,1.914697,"[{'name': 'Paramount Pictures', 'id': 4}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1932-12-08,25.0,89.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Every woman who has loved will understand,A Farewell to Arms,6.2,29.0
1465,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",105045,de,"East-Berlin, 1961, shortly after the erection ...",0.122178,"[{'name': 'Studio Babelsberg', 'id': 264}, {'n...","[{'iso_3166_1': 'DE', 'name': 'Germany'}]",1995-02-16,0.0,115.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}]",Released,"A love, a hope, a wall.",The Promise,5.0,1.0
2564,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",84198,en,"Using personal stories, this powerful document...",0.501046,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",2012-03-22,0.0,84.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,One Nation. Underfed.,A Place at the Table,6.9,7.0


In [528]:
df.iloc[45265]

belongs_to_collection                                                  NaN
budget                                                             5402000
genres                                      [{'id': 35, 'name': 'Comedy'}]
id                                                                  277839
original_language                                                       fr
overview                 Franky and Krimo dream of leaving the grey gri...
popularity                                                        5.613875
production_companies            [{'name': 'Mandarin Cinéma', 'id': 57294}]
production_countries              [{'iso_3166_1': 'FR', 'name': 'France'}]
release_date                                                    2016-02-24
revenue                                                                0.0
runtime                                                              100.0
spoken_languages                 [{'iso_639_1': 'fr', 'name': 'Français'}]
status                   

In [529]:
# Verificamos nulos de revenue y budget para reemplazamos por valor 0
df[['revenue','budget']].isnull().sum()

revenue    0
budget     0
dtype: int64

#### - Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [530]:
# verificamos la fila en donde se encuetra el dato errado => [] y lo reemplazamos por 0
# Armar luego una funcion para automatizarlo
df.iloc[19714]=0
df.iloc[29472]=0
df.iloc[35543]=0

In [531]:
df['revenue'].head(2)

0    373554033.0
1    262797249.0
Name: revenue, dtype: float64

In [532]:
# Pasamos los valores de la col 'budget' string a float. Tenemos 2 formas:
#df[['budget']].astype(float)
#df[['revenue']].astype(float)
#pd.to_numeric(df['budget'], downcast='float')
#pd.to_numeric(df['revenue'], downcast='float')

In [533]:
df['revenue']=df['revenue'].astype(float)

In [534]:
df['budget']=df['budget'].astype(float)

#### - Creamos la columna 'return' con la division de las columnas 'budget' y 'revenue'

In [535]:

df['return']=df['revenue'] / df['budget']

#### - Los valores nulos del campo 'release_date' deben eliminarse.

In [536]:
#Los valores nulos del campo release date deben eliminarse.
#df['release_date']=df['release_date'].dropna()
df = df.dropna(subset = ['release_date'])

In [537]:
df['release_date'].isnull().sum()

0

#### - Crear la columna 'release_year' donde extraerán el año de la fecha de estreno.

In [538]:
df.release_date

0        1995-10-30
1        1995-12-15
2        1995-12-22
3        1995-12-22
4        1995-02-10
            ...    
45460    1991-05-13
45462    2011-11-17
45463    2003-08-01
45464    1917-10-21
45465    2017-06-09
Name: release_date, Length: 45376, dtype: object

In [539]:
df.loc[19714]

belongs_to_collection      0
budget                   0.0
genres                     0
id                         0
original_language          0
overview                   0
popularity                 0
production_companies       0
production_countries       0
release_date               0
revenue                  0.0
runtime                  0.0
spoken_languages           0
status                     0
tagline                    0
title                      0
vote_average             0.0
vote_count               0.0
return                   NaN
Name: 19714, dtype: object

In [540]:
df.columns

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'return'],
      dtype='object')

In [541]:
# rellenamos los datos faltantes de la columna spoken_languages
df['spoken_languages']=df['spoken_languages'].fillna('Not Available')

#### - De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [462]:
# Cambiamos las fechas de la columna 'release_date' a formato AAAA-mm-dd, ignorando con 'coerce'
# los valores que no cumplen con el criterio para la funcion .to_datetime
df["release_date"] = pd.to_datetime(df['release_date'],errors='coerce')


In [463]:
df.release_date

0       1995-10-30
1       1995-12-15
2       1995-12-22
3       1995-12-22
4       1995-02-10
           ...    
45460   1991-05-13
45462   2011-11-17
45463   2003-08-01
45464   1917-10-21
45465   2017-06-09
Name: release_date, Length: 45376, dtype: datetime64[ns]

In [ ]:
# Crear una nueva columna 'year' con el año extraído
df['release_year'] = df['release_date'].dt.year.fillna(0).astype(int)
df.year

In [ ]:
# Crear una nueva columna 'release_month' con el año extraído
df['release_month'] = df['release_date'].dt.month.fillna(0).astype(int)


In [ ]:
df.release_month

In [ ]:
from datetime import datetime
import calendar

In [ ]:
# Obtener el nombre del día de la semana en inglés
df['release_day'] = df['release_date'].dt.day_name()

In [ ]:
# Rellenar los valores faltantes con una cadena vacía ('')
df['release_day'] = df['release_day'].fillna('')

In [ ]:
df.release_day

0          Monday
1          Friday
2          Friday
3          Friday
4          Friday
           ...   
45460      Monday
45462    Thursday
45463      Friday
45464      Sunday
45465      Friday
Name: release_day, Length: 45376, dtype: object

In [ ]:
#df['spoken_languages'][0:]
#ejemplo=pd.DataFrame(df['spoken_languages'])

In [ ]:
# ejemplo para dividir con coma','  la cadena de texto en donde aparece un espacio. '\s'
#re.split('\s',df['spoken_languages'][0])

In [ ]:
#valor_iso= df['spoken_languages'][1]
#print(re.search(r"'\w\w'",valor_iso))
# verificamos que los valores de la columna son cadena string.
#type(df['spoken_languages'][1])
#df_lenguaje= pd.DataFrame(df['spoken_languages'])
# alternativa para separar en nuevas columnas teniendo en cuenta como separador de la cadena string
# la coma ','
#df_lenguaje['spoken_languages'].str.split(',',n=3,expand=True)

In [ ]:
df['spoken_languages'].unique()

In [ ]:
import ast

def parse_spoken_languages(x):
    try:
        return ast.literal_eval(x)
    except (SyntaxError, ValueError):
        return []

df['spoken_languages'] = df['spoken_languages'].apply(lambda x: parse_spoken_languages(x) if pd.notnull(x) else [])

In [ ]:
df['spoken_languages'] = df['spoken_languages'].apply( lambda x: [lang['name'] for lang in x] if isinstance(x,list) else x)
df.spoken_languages

0                  [English]
1        [English, Français]
2                  [English]
3                  [English]
4                  [English]
                ...         
45460              [English]
45462                     []
45463              [English]
45464                     []
45465              [English]
Name: spoken_languages, Length: 45376, dtype: object

In [ ]:
# Otras opciones para parsear con regex
#df['iso_spoken_languages']=df.spoken_languages.str.extract("('\w\w')")
# Eliminar las comillas usando .str.strip()
#df['iso_spoken_languages'] = df['iso_spoken_languages'].str.strip("'")

In [ ]:
df.spoken_languages

0                  [English]
1        [English, Français]
2                  [English]
3                  [English]
4                  [English]
                ...         
45460              [English]
45462                     []
45463              [English]
45464                     []
45465              [English]
Name: spoken_languages, Length: 45376, dtype: object

In [ ]:
df.columns

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'return', 'release_year', 'release_day'],
      dtype='object')

In [ ]:
df.isnull().sum()

#### - Teniendo en cuenta las funciones a desarrollar, las siguientes columnas se consideran irrelevantes, ya sea por la falta de informacion o poco peso a la hora de evaluar las funciones. Las columnas son: 'belongs_to_collection', 'status', 'tagline', 'overview'.

In [ ]:
df.drop(['belongs_to_collection','status','tagline','overview'],axis=1,inplace=True)

In [ ]:
df.columns

In [ ]:
df.genres.unique()

In [ ]:
import ast

def parse_genres(x):
    try:
        return ast.literal_eval(x)
    except (SyntaxError, ValueError):
        return []

df['genres'] = df['genres'].apply(lambda x: parse_genres(x) if pd.notnull(x) else [])

# hemos creado una función parse_genres que intenta evaluar la cadena de texto utilizando ast.literal_eval(). 
# Si ocurre algún error durante la evaluación, como una sintaxis incorrecta o un valor no válido, la función 
# capturará la excepción y retornará una lista vacía en su lugar.
# Luego, aplicamos esta función a cada valor de la columna 'genres' utilizando apply(). 
# Si el valor es NaN, se asigna una lista vacía directamente.
# Con este enfoque, cualquier valor incorrecto o incompatible en la columna 'genres' se manejará adecuadamente 
# y se asignará una lista vacía en su lugar. Esto debería evitar que se produzca el error 
# "ValueError: malformed node or string: 0".


In [ ]:
df['genres'] = df['genres'].apply( lambda x: [lang['name'] for lang in x] if isinstance(x,list) else x)
df.genres

In [ ]:
df.columns

In [ ]:
# Utilizamos el mismo metodo que 'genres' (para columnas anidadas)
def parse_production_companies(x):
    try:
        return ast.literal_eval(x)
    except (SyntaxError, ValueError):
        return []

df['production_companies'] = df['production_companies'].apply(lambda x: parse_production_companies(x) if pd.notnull(x) else [])

In [ ]:
df['production_companies'] = df['production_companies'].apply( lambda x: [lang['name'] for lang in x] if isinstance(x,list) else x)
df.production_companies

In [ ]:
# Utilizamos el mismo metodo que 'genres' (para columnas anidadas)
def parse_production_countries(x):
    try:
        return ast.literal_eval(x)
    except (SyntaxError, ValueError):
        return []

df['production_countries'] = df['production_countries'].apply(lambda x: parse_production_countries(x) if pd.notnull(x) else [])

In [ ]:
df['production_countries'] = df['production_countries'].apply( lambda x: [lang['iso_3166_1'] for lang in x] if isinstance(x,list) else x)
df.production_countries

In [ ]:
df_v1=df

In [466]:
df_v1.columns

Index(['budget', 'genres', 'id', 'original_language', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'title', 'vote_average',
       'vote_count', 'return', 'release_year', 'release_day'],
      dtype='object')

In [471]:
# opcion 1 -Pasamos los codigoas a uppercase. utilizamos el método str.upper() para convertir los códigos ISO a mayúsculas y el 
# método tolist()  para convertir la columna en una lista.
df_v1['original_language']=df_v1['original_language'].str.upper().tolist()
# opcion 2 -Convertir los códigos ISO a mayúsculas y agregar corchetes
df['original_language'] = df['original_language'].apply(lambda x: f"[{x.upper()}]")


In [486]:
# Pasamos float a int
df_v1['vote_count']=df_v1.vote_count.astype(int)

In [494]:
# Pasamos object to int
df_v1['id']=df_v1.id.astype(int)

In [505]:
df_v1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45376 entries, 0 to 45465
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   budget                45376 non-null  float64       
 1   genres                45376 non-null  object        
 2   id                    45376 non-null  int32         
 3   original_language     45376 non-null  object        
 4   popularity            45376 non-null  object        
 5   production_companies  45376 non-null  object        
 6   production_countries  45376 non-null  object        
 7   release_date          45376 non-null  datetime64[ns]
 8   revenue               45376 non-null  float64       
 9   runtime               45376 non-null  float64       
 10  spoken_languages      45376 non-null  object        
 11  title                 45376 non-null  object        
 12  vote_average          45376 non-null  float64       
 13  vote_count           

In [504]:
df_v1['release_date'].fillna('1900-01-01',inplace=True)

In [510]:
df_v1 = df_v1.drop_duplicates(subset='id')

In [511]:
df_v1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45344 entries, 0 to 45465
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   budget                45344 non-null  float64       
 1   genres                45344 non-null  object        
 2   id                    45344 non-null  int32         
 3   original_language     45344 non-null  object        
 4   popularity            45344 non-null  object        
 5   production_companies  45344 non-null  object        
 6   production_countries  45344 non-null  object        
 7   release_date          45344 non-null  datetime64[ns]
 8   revenue               45344 non-null  float64       
 9   runtime               45344 non-null  float64       
 10  spoken_languages      45344 non-null  object        
 11  title                 45344 non-null  object        
 12  vote_average          45344 non-null  float64       
 13  vote_count           

In [513]:
df_v1.to_csv('ETL_v1_Movies.csv',index=False)